# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [ ]:
import os

# Set your preferred name
USER = "Shuyi"
# Set the data and output directories
DATA_ROOT = '/content/drive/MyDrive/AI4Science_Public/data'
OUTPUT_ROOT = '/content/drive/MyDrive/AI4Science_Public/output1'
TRAIT = "Underweight"

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [ ]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)

Mounted at /content/drive


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
!pip install mygene
!pip install sparse_lmm

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [ ]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['.DS_Store',
 'TCGA_Esophageal_Cancer_(ESCA)',
 'TCGA_Pheochromocytoma_Paraganglioma_(PCPG)',
 'TCGA_Bile_Duct_Cancer_(CHOL)',
 'TCGA_Kidney_Chromophobe_(KICH)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Sarcoma_(SARC)',
 'TCGA_Adrenocortical_Cancer_(ACC)',
 'TCGA_Thymoma_(THYM)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [ ]:
trait_subdir = "TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.GBMLGG.sampleMap_GBMLGG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.GBMLGG.sampleMap_HiSeqV2_PANCAN.gz')

In [ ]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PANCAN_CNA_PANCAN_K8',
 '_PANCAN_Cluster_Cluster_PANCAN',
 '_PANCAN_DNAMethyl_PANCAN',
 '_PANCAN_RPPA_PANCAN_K8',
 '_PANCAN_UNC_RNAseq_PANCAN_K16',
 '_PANCAN_mutation_PANCAN',
 '_PATIENT',
 '_cohort',
 '_primary_disease']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'additional_surgery_metastatic_procedure', 'age_at_initial_pathologic_diagnosis', 'animal_insect_allergy_history',

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'days_to_performance_status_assessment', 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust', 'first_diagnosis_age_of_animal_insect_allergy', 'first_diagnosis_age_of_food_allergy', 'year_of_initial_pathologic_diagnosis']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [44.0, 50.0, 59.0, 56.0, 40.0],
 'days_to_birth': [-16179.0, -18341.0, -21617.0, -20516.0, -14806.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': ['358', '144', '345', '558', '705'],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': [279.0, 144.0, 345.0, 558.0, 705.0],
 'days_to_new_tumor_event_additional_surgery_procedure': [nan,
  nan,
  nan,
  nan,
  nan],
 'days_to_new_tumor_event_after_initial_treatment': ['137',
  '40',
  '314',
  '302',
  '518'],
 'days_to_performance_status_assessment': [nan, nan, nan, nan, nan],
 'first_diagnosis_age_asth_ecz_hay_fev_mold_dust': [nan, nan, nan, nan, nan],
 'first_diagnosis_age_of_animal_insect_allergy': [nan, nan, nan, nan, nan],
 'first_diagnosis_age_of_food_allergy': [nan, nan, nan, nan, nan],
 'year_of_initial_pathologic_diagnosis': [2002.0,
  2003.0,
  2002.0,
  2002.0,
  2002.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'MALE', 'FEMALE', 'FEMALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lower Grade Glioma,Age,Gender,UBE2Q1,RNF14,SERPINB11,RNF17,RNF10,RNF11,RNF13,...,SLC7A10,SLC7A11,FOLR1,TULP2,NPY5R,GOLGA6D,SERPINH1,NFIX,SELP,ADAMTS19
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-02-0047-01,1,78.0,1.0,0.344382,4.50804,-0.640695,-9.802658,0.334599,0.935065,-0.392218,...,3.350214,8.265318,-2.369297,-0.748878,3.893183,-1.06467,0.104503,4.740464,-8.884833,-2.628884
TCGA-02-0055-01,1,62.0,0.0,0.842282,4.59894,-0.640695,-9.802658,0.341599,1.272965,-1.092218,...,2.018414,7.854518,0.102403,0.674422,-0.945017,-0.42257,1.094003,2.900964,-5.181933,-3.338484
TCGA-02-2483-01,1,43.0,1.0,0.973682,3.86694,-0.640695,-9.802658,0.567399,1.409265,-0.986218,...,-0.661586,6.404818,-5.283397,-0.748878,-1.079017,-1.06467,-0.136297,4.453664,-8.860933,-4.527484
TCGA-02-2485-01,1,53.0,1.0,0.433582,4.09604,-0.640695,-9.802658,0.314499,0.949965,-1.107318,...,5.182814,7.843218,-4.165997,-0.748878,-0.621917,-0.50277,-0.569697,5.598364,-8.761333,-5.035584
TCGA-02-2486-01,1,64.0,1.0,0.554782,4.89974,-0.640695,-9.802658,0.828999,1.160065,-0.123618,...,-0.574886,7.754918,0.030803,2.594022,-0.900217,-1.06467,-0.877297,4.014264,-8.891233,-5.035584


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 695 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lower Grade Glioma', the least common label is '1' with 695 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Lower Grade Glioma' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 59.0
Min: 14.0
Max: 89.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 297 occurrences. This represents 42.73% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



True

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [ ]:
dataset = 'GEO'
trait_subdir = "Underweight"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE84954',
 'GSE57802',
 'GSE85017',
 'GSE163901',
 'GSE163902',
 'GSE50982',
 'GSE163903',
 'GSE119483',
 'GSE130563',
 'GSE131835',
 'GSE10135']

Repeat the below steps for all the accession numbers

In [ ]:
cohort = accession_num = "GSE10135"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/content/drive/MyDrive/AI4Science_Public/data/GEO/Underweight/GSE10135/GSE10135_family.soft.gz',
 '/content/drive/MyDrive/AI4Science_Public/data/GEO/Underweight/GSE10135/GSE10135-GPL80_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [ ]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Cigarette Smoking Induces Overexpression of a Fat Depleting Gene AZGP1 in the Human Airway Epithelium"
!Series_summary	"Smokers weigh less and have less body fat than non-smokers, and increased body fat and weight gain are observed following smoking cessation. To assess a possible molecular mechanism underlying the inverse association between smoking and body weight, we hypothesized that smoking may induce the expression of a fat depleting gene in the airway epithelium, the cell population that takes the brunt of the stress of cigarette smoke. As a candidate gene we evaluated the expression of alpha2-zinc-glycoprotein1 (AZGP1), a soluble protein that stimulates lipolysis, induces a reduction in body fat in mice and is associated with the cachexia related to cancer, and is known to be expressed in secretory cells of lung epithelium. To assess if smoking upregulates AZGP1 expression, microarray analysis with TaqMan confirmation was used to evaluate large airway epithelial 

In [ ]:
clinical_data.head()

,!Sample_geo_accession,GSM114035,GSM114037,GSM114039,GSM114041,GSM114043,GSM114045,GSM114047,GSM114049,GSM114051,...,GSM114059,GSM114061,GSM114063,GSM114065,GSM114067,GSM114069,GSM114071,GSM114073,GSM114075,GSM114077
0,!Sample_characteristics_ch1,Age: 33,Age: 24,Age: 48,Age: 57,Age: 57,Age: 36,Age: 39,Age: 32,Age: 22,...,Age: 37,Age: 31,Age: 39,Age: 25,Age: 27,Age: 38,Age: 47,Age: 33,Age: 43,Age: 41
1,!Sample_characteristics_ch1,Sex: M,Sex: M,Sex: M,Sex: F,Sex: M,Sex: M,Sex: M,Sex: F,Sex: M,...,Sex: M,Sex: M,Sex: M,Sex: M,Sex: F,Sex: M,Sex: M,Sex: F,Sex: F,Sex: M
2,!Sample_characteristics_ch1,Ethnic group: white,Ethnic group: black,Ethnic group: black,Ethnic group: white,Ethnic group: white,Ethnic group: white,Ethnic group: black,Ethnic group: black,Ethnic group: black,...,Ethnic group: white,Ethnic group: white,Ethnic group: white,Ethnic group: white,Ethnic group: black,Ethnic group: black,Ethnic group: white,Ethnic group: black,Ethnic group: white,Ethnic group: black
3,!Sample_characteristics_ch1,Smoking Status: non-smoker,Smoking Status: non-smoker,Smoking Status: non-smoker,Smoking Status: non-smoker,Smoking Status: non-smoker,Smoking Status: non-smoker,Smoking Status: non-smoker,Smoking Status: non-smoker,Smoking Status: non-smoker,...,"Smoking Status: smoker, 20 pack-years","Smoking Status: smoker, 15 pack-years","Smoking Status: smoker, 24 pack-years","Smoking Status: smoker, 10 pack-years","Smoking Status: smoker, 8 pack-years","Smoking Status: smoker, 14 pack-years","Smoking Status: smoker, 28 pack-years","Smoking Status: smoker, 19 pack-years","Smoking Status: smoker, 38 pack-years","Smoking Status: smoker, 20 pack-years"
4,!Sample_characteristics_ch1,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,...,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right,Lung: right


In [ ]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['Age: 33',
  'Age: 24',
  'Age: 48',
  'Age: 57',
  'Age: 36',
  'Age: 39',
  'Age: 32',
  'Age: 22',
  'Age: 41',
  'Age: 55',
  'Age: 37',
  'Age: 31',
  'Age: 25',
  'Age: 27',
  'Age: 38',
  'Age: 47',
  'Age: 43'],
 1: ['Sex: M', 'Sex: F'],
 2: ['Ethnic group: white', 'Ethnic group: black', 'Ethnic group: hispanic'],
 3: ['Smoking Status: non-smoker',
  'Smoking Status: smoker, 12 pack-years',
  'Smoking Status: smoker, 30 pack-years',
  'Smoking Status: smoker, 20 pack-years',
  'Smoking Status: smoker, 15 pack-years',
  'Smoking Status: smoker, 24 pack-years',
  'Smoking Status: smoker, 10 pack-years',
  'Smoking Status: smoker, 8 pack-years',
  'Smoking Status: smoker, 14 pack-years',
  'Smoking Status: smoker, 28 pack-years',
  'Smoking Status: smoker, 19 pack-years',
  'Smoking Status: smoker, 38 pack-years'],
 4: ['Lung: right']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [ ]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Underweight\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Underweight\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an in

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [ ]:
is_gene_availabe = True
trait_row_id = None
age_row_id = 0
gender_row_id = 1

trait_type = 'binary'

In [ ]:
is_available = is_gene_availabe and (trait_row_id is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [ ]:
# Verify and use the functions generated by GPT

def convert_trait(value):
    if value == 'group: CWL':
        return '1'
    else:
        return '0'

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None


def convert_gender(value):
    if value == 'Sex: Female':
        return '1'
    elif value == 'Sex: Male':
        return '0'
    else:
        return None


In [ ]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row_id, age_row_id, gender_row_id,
                                                      convert_trait, convert_age, convert_gender)
selected_clinical_data.head()

,GSM3821249,GSM3821251,GSM3821252,GSM3821254,GSM3821255,GSM3821257,GSM3821258,GSM3821259,GSM3821261,GSM3821262,...,GSM3821304,GSM3821305,GSM3821307,GSM3821308,GSM3821309,GSM3821311,GSM3821312,GSM3821314,GSM3821315,GSM3821316
Underweight,0,0,0,1,0,0,0,0,0,0,...,1,1,0,0,1,0,1,1,1,1
Age,51.0,64.0,62.0,78.0,47.0,59.0,51.0,57.0,47.0,58.0,...,81.0,65.0,65.0,65.0,72.0,64.0,68.0,72.0,59.0,59.0
Gender,0,0,1,1,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [ ]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,ID,GSM3821249,GSM3821251,GSM3821252,GSM3821254,GSM3821255,GSM3821257,GSM3821258,GSM3821259,GSM3821261,...,GSM3821304,GSM3821305,GSM3821307,GSM3821308,GSM3821309,GSM3821311,GSM3821312,GSM3821314,GSM3821315,GSM3821316
0,ENSG00000000003_at,1.372046,1.452475,1.328127,1.218645,1.063923,1.421831,1.623738,1.571429,1.437436,...,1.280086,1.462220,1.357814,1.404662,1.132363,1.541602,1.258185,1.247770,1.359168,1.149634
1,ENSG00000000005_at,0.503409,0.366208,0.522604,0.894651,1.488323,0.576856,1.052436,1.575757,1.196918,...,0.533244,0.544249,0.677682,0.867532,0.545122,1.684016,0.282615,1.041511,1.171564,1.259981
2,ENSG00000000419_at,1.426808,1.484094,1.328433,1.379408,1.482836,1.492530,1.365101,1.310677,1.472370,...,1.594919,1.456356,1.571216,1.591026,1.590308,1.325737,1.605550,1.561210,0.965948,1.553152
3,ENSG00000000457_at,0.851691,0.732859,0.560948,0.798012,0.597443,0.707084,0.728770,0.679633,0.722328,...,0.669531,0.849198,0.706279,0.783585,0.835467,0.677518,0.829129,0.706056,0.433194,0.706191
4,ENSG00000000460_at,0.188718,0.247585,0.370477,0.011395,0.269866,0.514771,0.222136,0.413359,0.278311,...,0.332589,0.366151,0.338951,0.253835,0.022019,0.361548,0.184171,0.012481,0.012370,0.020309


In [ ]:
gene_row_ids = genetic_data['ID'][:20].tolist()
gene_row_ids

['ENSG00000000003_at',
 'ENSG00000000005_at',
 'ENSG00000000419_at',
 'ENSG00000000457_at',
 'ENSG00000000460_at',
 'ENSG00000000938_at',
 'ENSG00000000971_at',
 'ENSG00000001036_at',
 'ENSG00000001084_at',
 'ENSG00000001167_at',
 'ENSG00000001460_at',
 'ENSG00000001461_at',
 'ENSG00000001497_at',
 'ENSG00000001561_at',
 'ENSG00000001617_at',
 'ENSG00000001626_at',
 'ENSG00000001629_at',
 'ENSG00000001631_at',
 'ENSG00000002016_at',
 'ENSG00000002330_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [ ]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at', 'ENSG00000000457_at', 'ENSG00000000460_at', 'ENSG00000000938_at', 'ENSG00000000971_at', 'ENSG00000001036_at', 'ENSG00000001084_at', 'ENSG00000001167_at', 'ENSG00000001460_at', 'ENSG00000001461_at', 'ENSG00000001497_at', 'ENSG00000001561_at', 'ENSG00000001617_at', 'ENSG00000001626_at', 'ENSG00000001629_at', 'ENSG00000001631_at', 'ENSG00000002016_at', 'ENSG00000002330_at']\n"

If not required, jump directly to the gene normalization step

In [ ]:
requires_gene_mapping = True

In [ ]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)[0]
    gene_annotation_summary = preview_df(gene_annotation)
    gene_annotation_summary

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [ ]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = key_name1
    gene_name_key = key_name2

    Gene annotation dictionary:
    {'ID': ['ENSG00000000003_at', 'ENSG00000000005_at', 'ENSG00000000419_at', 'ENSG00000000457_at', 'ENSG00000000460_at'], 'Ensembl Gene ID': ['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460'], 'HGNC ID': ['HGNC:11858', 'HGNC:17757', 'HGNC:3005', 'HGNC:19285', 'HGNC:25565'], 'ENTREZ_GENE_ID': [7105.0, 64102.0, 8813.0, 57147.0, 55732.0], 'ORF': ['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112'], 'SPOT_ID': ['ENSG00000000003', 'ENSG00000000005', 'ENSG00000000419', 'ENSG00000000457', 'ENSG00000000460']}
    


In [ ]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'ORF'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of duplicate or missing query terms.
INFO:biothings.client:querying 1-1000...
INFO:biothings.client:done.
INFO:biothings.client:Finished.
INFO:biothings.client:Pass "returnall=True" to return complete lists of

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 48 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Underweight', the least common label is '1' with 16 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Underweight' in this dataset is fine.

Quartiles for 'Age':
  25%: 53.75
  50% (Median): 59.0
  75%: 65.0
Min: 41.0
Max: 81.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 18 occurrences. This represents 37.50% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE131835'

In [ ]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
9,GSE131835,True,True,False,True,True,48.0,


In [ ]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Underweight,Age,Gender,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
0,0,51.0,0,-0.264651,2.610365,0.021361,-0.241874,0.410958,-0.217128,1.360521,...,-0.076491,0.123921,0.034939,0.132122,1.065709,-0.072836,1.033788,1.174154,0.590178,1.089585
1,0,64.0,0,-0.272516,2.480320,0.045095,-0.229687,0.430536,-0.138089,1.569791,...,-0.057494,0.038545,0.195941,0.304629,1.177111,-0.165188,1.247287,1.084549,0.563419,1.008246
2,0,62.0,1,-0.103764,2.540219,0.034681,-0.141080,0.182214,-0.162489,0.986534,...,-0.203573,0.048044,-0.024995,-0.070301,0.815424,0.237715,0.823749,0.626423,0.397357,0.570412
3,1,78.0,1,-0.202635,2.760215,0.091284,-0.262594,0.340630,-0.140353,1.376508,...,-0.309146,0.051332,0.207470,0.083145,1.206411,-0.003740,0.919370,0.918313,0.213342,0.822023
4,0,47.0,0,-0.142491,2.754827,0.123384,-0.159299,0.421173,-0.274787,1.288496,...,-0.045949,-0.031918,0.084441,0.140668,1.230014,-0.056530,1.147231,0.978320,0.437819,1.049170


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 53.33% ± 16.33%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 66.67% ± 12.17%


In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 5. Discussion and report

In [ ]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.3134
Found 37 genes with non-zero coefficients associated with the trait 'Underweight' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
 Variable  Coefficient  Absolute Coefficient
 SH3BGRL2    12.003568             12.003568
   PTPN11     6.002774              6.002774
   COX7B2    -5.248039              5.248039
  BCL2L12     3.985842              3.985842
    HYAL4    -3.807759              3.807759
  SELENOW     3.430017              3.430017
  ATXN7L2     3.097979              3.097979
    OR1L1    -2.589730              2.589730
     ZBBX    -2.508281              2.508281
SUPT20HL2     2.196019              2.196019
   ELOVL4    -2.048656              2.048656
    LMCD1     2.024814              2.024814
  MKRN2OS     2.021444              2.021444
    OR2L8    -1.893193              1.893193
     THBD     1.795917              1.795917
LINC01567 

/content/drive/MyDrive/Colab Notebooks/utils.py:434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.3134
Found 0 significant genes associated with the trait 'Underweight' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/content/drive/MyDrive/Colab Notebooks/utils.py:443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
